In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
from datetime import datetime
from datetime import datetime
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
import random
import sys

In [ ]:
# Import data
flu = pd.read_csv('../data/INFLUENZA_sentinella/data.csv')
weather = pd.read_csv('../data/weather/reg_weather.csv')
google_flu = pd.read_csv('../data/google_search_trend/reg_google_grippe.csv')
google_symptoms = pd.read_csv('../data/google_search_trend/reg_google_fieber_husten.csv')
pop = pd.read_csv('../data/pop_data_cantons/weekly_imputed_pop_data_final.csv')

<h2>Data Consolidation</h2>

<h4>Extract relevant data from BAG dataset on weekly flu incidence</h4>

In [ ]:
# Create dataframe for regional observations, no differentiation between sex or age
flu_reg = flu.query('georegion_type == "sentinella_region" and agegroup == "all" and sex == "all"').copy()

# Drop rows for georegion "unknown", which only contain NaNs using mask
flu_reg = flu_reg[~(flu_reg['georegion'] == 'unknown')]

# Select columns required for analysis
selected_cols = ['temporal', 'georegion', 'incValue', 'value']
flu_reg = flu_reg[selected_cols]

<h4>Convert dates and format of Google-Trend data for subsequent merging</h4>

In [ ]:
# Align time-indexes of google data and flu data using created date to iso-calendar week dict
with open('date_dict.json', 'r') as f:
    date_dict = json.load(f)

# Create new column 'Woche' containing iso-calendar weeks for google-trend dates 
google_flu['Woche'] = google_flu['Woche'].apply(lambda x: date_dict[x]) 
google_symptoms['Woche'] = google_symptoms['Woche'].apply(lambda x: date_dict[x])

In [ ]:
# Reshape google_flu from wide to long to enable merging on date and region 
google_flu = google_flu.melt(id_vars=['Woche'], var_name='region_query', value_name='search_activity')

# Separate region and query information from header into separate rows
google_flu['region'] = google_flu['region_query'].apply(lambda x: "_".join(x.split('_')[:2]))
google_flu['query'] = google_flu['region_query'].apply(lambda x: "_".join(x.split('_')[2:]))
google_flu.drop(columns='region_query', inplace=True) # Drop superfluous region_query column

# Reshape dataframe to get separate columns for each variable
google_flu = google_flu.pivot(index=['Woche', 'region'], columns='query', values='search_activity').reset_index()

In [ ]:
## Repeat above process for second google dataset containing data on symptom queries
# Reshape google_flu from wide to long
google_symptoms = google_symptoms.melt(id_vars=['Woche'], var_name='region_query', value_name='search_activity')

# Separate region and query information from header into separate rows
google_symptoms['region'] = google_symptoms['region_query'].apply(lambda x: "_".join(x.split('_')[:2]))
google_symptoms['query'] = google_symptoms['region_query'].apply(lambda x: "_".join(x.split('_')[2:]))
google_symptoms.drop(columns='region_query', inplace=True) # Drop superfluous region_query column

# Reshape dataframe to get separate columns for each variable
google_symptoms = google_symptoms.pivot(index=['Woche', 'region'], columns='query', values='search_activity').reset_index()

<h4>Convert date format of weather data for merging</h4>

In [ ]:
# Convert dates to 'YYYY-Www' ISO week format
dates = weather.date.values
iso_week_dates = [datetime.strptime(date, '%Y-%m-%d').isocalendar()[:2] for date in dates]
iso_week_dates = [f'{year}-W{week:02d}' for year, week in iso_week_dates]
weather['date'] = iso_week_dates

<h4>Merge datasets on date and region</h4>

In [ ]:
merged_data = pd.merge(flu_reg, weather, how='left', left_on=['temporal', 'georegion'], right_on=['date', 'region']).sort_values(by=['georegion', 'temporal'])
merged_google = pd.merge(google_flu, google_symptoms, how='inner', on=['region', 'Woche'])
merged_data = pd.merge(merged_data, merged_google, how='left', left_on=['georegion', 'temporal'], right_on=['region', 'Woche'])

In [ ]:
merged_data.drop(columns=['region_x', 'region_y', 'date', 'Woche'], inplace=True)

<h4>Convert ISO calendar-weeks to Gregorian calendar (format 'YYYY-MM-DD')</h4>

In [ ]:
import datetime

# Convert ISO calendar-weeks to gregorian dates
# Functions based on answer by Ben James: <https://stackoverflow.com/questions/304256/whats-the-best-way-to-find-the-inverse-of-datetime-isocalendar>
def iso_year_start(iso_year):
    "The gregorian calendar date of the first day of the given ISO year"
    fourth_jan = datetime.date(iso_year, 1, 4)
    delta = datetime.timedelta(fourth_jan.isoweekday()-1)
    return fourth_jan - delta 

def iso_to_gregorian(iso_year, iso_week, iso_day):
    "Gregorian calendar date for the given ISO year, week and day"
    year_start = iso_year_start(iso_year)
    return year_start + datetime.timedelta(days=iso_day-1, weeks=iso_week-1)


In [ ]:
# Extract week number and year from date column in ISO calendar week 
week_pattern = r'W(\d{1,2})' # RegEx pattern to extract week nr without trailing zero
merged_data['week_number'] = merged_data['temporal'].str.extract(week_pattern).astype(int)
merged_data['year'] = merged_data['temporal'].apply(lambda x: x.split('-')[0])
merged_data['year'] = pd.to_numeric(merged_data['year']) # Convert from string to numeric

# Convert from iso-calendar week to gregorian dates (format: YYYY-MM-DD)
merged_data['date'] = list(map(lambda year, week: iso_to_gregorian(year, week, 4), merged_data['year'], merged_data['week_number']))

In [ ]:
# Inspect resulting dataframe
merged_data.head()

<h2>Data exploration</h2>

In [ ]:
# Display descriptive statistics
merged_data['incValue'].describe()

In [ ]:
# Display missing values in reported flu incidence across regions
merged_data[merged_data['incValue'].isna()]

In [ ]:
# Impute missing values in March 2020 linearly
merged_data['incValue'].interpolate(inplace=True)

<h4>Inspect incidence of consultations for influenza-like-diseases over time</h4>

In [ ]:
# fig, ax = plt.subplots(6, figsize=(12, 12))
# for i in range(1, 7):
#     ax[i-1].plot(merged_data.loc[merged_data['georegion'] == f"region_{i}", 'incValue'])
#     ax[i-1].plot(merged_data.loc[merged_data['georegion'] == f"region_{i}", 'Grippe'])

import matplotlib.pyplot as plt

# Dictionary for cantons within each region
region_to_ct = {'region_1': ['Genf', 'Neuenburg', 'Waadt', 'Wallis'], 
           'region_2': ['Bern', 'Freiburg', 'Jura'], 
           'region_3': ['Aargau', 'Basel-Landschaft', 'Basel-Stadt', 'Solothurn'], 
           'region_4': ['Luzern', 'Nidwalden', 'Obwalden', 'Schwyz', 'Uri', 'Zug'], 
           'region_5': ['Appenzell_Innerrhoden', 'Appenzell_Ausserrhoden', 'Glarus', 'Sankt_Gallen', 'Schaffhausen', 'Thurgau', 'Zürich'], 
           'region_6': ['Graubünden', 'Tessin']}

# Assuming merged_data is your DataFrame and it's already been imported.
plt.style.use('ggplot')
fig, ax = plt.subplots(6, figsize=(10, 12))

for i in range(1, 7):
    # Filter the data for the current region
    region_data = merged_data[merged_data['georegion'] == f"region_{i}"]
    
    # Find the maximum values for 'incValue' and 'Grippe' in the current region
    max_incValue_region = region_data['incValue'].max()
    max_Grippe_region = region_data['Grippe'].max()
    
    # Determine the scale factors
    scale_factor_incValue = 1.0 / max_incValue_region if max_incValue_region != 0 else 1
    scale_factor_Grippe = 1.0 / max_Grippe_region if max_Grippe_region != 0 else 1
    
    # Apply the scale factors to normalize the amplitude
    normalized_incValue = region_data['incValue'] * scale_factor_incValue
    normalized_Grippe = region_data['Grippe'] * scale_factor_Grippe
    
    # Plot the normalized data
    ax[i-1].plot(normalized_incValue, alpha=.8)
    ax[i-1].plot(normalized_Grippe, color='blue', alpha=.8)
    ax[i-1].set_title(f'Region {i}: {region_to_ct[f"region_{i}"]}', fontsize=10)

plt.suptitle('Normalized incValue and Google Search Activity per Region for the time frame between 2013 and late 2023')
plt.tight_layout()
plt.show()



<h2>Modelling</h2>

As the above plot demonstrates, google search search results for the term 'Grippe' follow the incidence of consultations for influenza-like-diseases quite closely. However, in the regions with less population, such as region 4 and 6, the quality of the data suffers dramatically. The following model attempts to refine the predictions from the purely autoregressive neural network by also considerung varying ranges of lags for the search term 'Grippe' in region 5. 

In order to do so, the following functions are required:

* create_lagged_features: Outputs a dataframe with columns at a specified range of lags given a dictionary specifying the column name and the number of lags to create for that column. 

In [ ]:
import pandas as pd

def create_lagged_features(df, column_lags, seasonal_lags=None):
    """
    Create lagged features for multiple columns in a DataFrame, with a specified number of lags for each column.

    Parameters:
    df (DataFrame): The original DataFrame.
    columns_with_lags (dict): A dictionary mapping column names to the number of lags. e.g., {'column1': 3, 'column2': 5}
    seasonal_lags (list of int, optional): Additional seasonal lags to include.

    Returns:
    DataFrame: A DataFrame containing the original columns and their lagged features.
    """

    df_lagged = df.copy()
    seasonal_lags = seasonal_lags or []

    for column, num_lags in column_lags.items():
        # Create specified number of lagged features for each column
        for lag in range(1, num_lags + 1):
            df_lagged[f'{column}_lag_{lag}'] = df_lagged[column].shift(lag)

        # Create seasonal lagged features if specified
        for seasonal_lag in seasonal_lags:
            df_lagged[f'{column}_seasonal_lag_{seasonal_lag}_helper'] = df_lagged[column].shift(seasonal_lag-1)
            df_lagged[f'{column}_seasonal_lag_{seasonal_lag}'] = df_lagged[column].shift(seasonal_lag)

    return df_lagged


* iterative_forecast: Perform iterative one-step-ahead forecasting using a pretrained model for a specified number of steps given initial input of lag observations and an array of forecasts for the exogenous variable for the length of the forecast (created separately using function "create_exogenous_input" below).

In [ ]:
def iterative_forecast(model, initial_input, seasonal_input,  n_steps, exogenous_forecasts=None):
    """
    Perform iterative forecasting using a pretrained model with optional exogenous variable forecasts.

    Args:
        model: Trained autoregressive model (e.g., MLPRegressor).
        initial_input: The initial input features for the target variable (e.g., the last observation from the training set).
        seasonal_input: Seasonal input features for the target variable.
        exogenous_forecasts: (Optional) List or array containing the forecasted values for the exogenous variable(s) for each future time step.
        n_steps: Number of future time steps to forecast.

    Returns:
        A list of forecasts, one for each future time step.
    """
    # Initiate empty list for forecasts
    forecasts = []

    # Transform inputs into np.arrays
    current_input = np.array(initial_input)
    seasonal_input = np.array(seasonal_input)

    # Check if exogenous forecasts are provided
    if exogenous_forecasts is None:
        exogenous_forecasts = np.empty((n_steps, 0))  # Empty array with 0 columns

    for i in range(n_steps):
        # Prepare the input array for prediction
        combined_input = np.concatenate([
            current_input,
            exogenous_forecasts[i] if exogenous_forecasts.shape[1] > 0 else []  # Add the forecasted exogenous values if provided
        ])
        
        # Predict the next step based on the combined input
        next_step_pred = model.predict(combined_input.reshape(1, -1))[0]
        forecasts.append(next_step_pred)

        # Update the autoregressive lags based on the forecast (next_step_pred becomes most recent lag)
        current_input = np.roll(current_input, 1)
        current_input[0] = next_step_pred
        
        # Update the seasonal lag based on true values for the first year and the forecasted values from then on
        if i < 52:
            current_input[-1] = seasonal_input[i]
        else:
            current_input[-1] = forecasts[i - 52]

    return np.array(forecasts)

* create_exogenous_input: Based on a separate forecasting model (in this case the same architecture for the google data) predict the required timeframe ahead for the exogenous variable and output an array containing forecasts for the lags

In [ ]:
def create_exogenous_input(model, initial_input, seasonal_input, n_steps):
    """
    Create a 2D array for exogenous variable inputs for each forecast step.

    Args:
        model: Model for exogenous variable to predict one week ahead.
        initial_input (array-like): The initial lag values for the exogenous variable.
        seasonal_input (array-like): Array of the values one week before the seasonal column for updating (essentially the lag_51 column, named 'seasonal_helper' in this notebook).
        n_steps (int): Number of future time steps to forecast.

    Returns:
        numpy.ndarray: A 2D array where each row represents all the lags and the seasonal lag for the exogenous variable for each week.
    """
    num_lags = len(initial_input)
    
    initial_input = np.array(initial_input)
    seasonal_input = np.array(seasonal_input)
    
    output_array = np.zeros((n_steps, num_lags))

    # Set initial lags and seasonal lag
    output_array[0, :] = initial_input

    forecasts = []
    for i in range(1, n_steps):
        # Predict the next step
        next_step_pred = model.predict(output_array[i-1,:].reshape(1, -1))[0]
        forecasts.append(next_step_pred)

        # Update lags: shift left and add new forecast
        output_array[i, :-1] = np.roll(output_array[i-1, :-1], 1) # Roll the consecutive autoregressive lags from one row before to shift weeks
        output_array[i, 0] = next_step_pred  # Insert most recent forecast in first position

        # Update seasonal lag
        if i < 52:
            output_array[i, -1] = seasonal_input[i-1]
        else:
            output_array[i, -1] = output_array[i-52, 0] # Fill in using the forecast from 52 weeks ago

    return output_array # Don't return the first column where value are all known to avoid shift

<h2>Model Tuning</h2>

In the following model, we're attempting to introduce Google search trend data for the query 'Grippe' (the German word for flu) for greater predictive power.

The model encapsulates a second model which forecasts the exogenous variable for the demanded timeframe. The configuration of the second model has been tuned separately and is also a feedforward neural network in this case (see notebook "ffn_exogenous.ipynb"). Based on the predicted exogenous features and its own initial autoregressive lags, the model then forecasts the incidence in an iterative procedure, predicting one-step ahead and using this as the most recent lag for the next step. An alternative to such iterative forecasting would be to directly model multiple steps ahead. Such methods couldn't be considered due to time constraints but should be explored in continued research.

In [ ]:
# Extract data
data = merged_data.loc[(merged_data['georegion'] == "region_5") & (merged_data['date'].apply(lambda x: x.year < 2020))]
# Split the data
y = data['incValue']
split = int(len(y) * 0.8)
y_train, y_test = y[:split], y[split:]

i = 0
scores_df = pd.DataFrame(columns=['RMSE', 'lags', 'seasonal_lags', 'hidden_layers', 'alpha', 'batch_size', 'activation', 'learning_rate'])

In [ ]:
# Suppress convergence warnings
# warnings.filterwarnings('ignore', category=ConvergenceWarning)

# Define parameter configurations to assess
lags = 52 # Autoregressive lags to consider
hidden_layer_sizes = [(16, 16), (16, 16, 16)]
alphas = np.linspace(0.01, 0.3, num=100) # Regularization parameter
batch_size = 32
learning_rates = np.logspace(-3, -4, 100)
activations = ['relu']
seasonal = [52]
exog_lags = [21]

models_count = (lags-44) * 18 * len(hidden_layer_sizes) * len(alphas) * len(learning_rates) * len(activations)

# Filter the DataFrame
data = merged_data.loc[
    (merged_data['georegion'] == "region_5") & 
    (merged_data['date'].apply(lambda x: (x.year < 2020))) & 
    (merged_data['date'].apply(lambda x: (x > datetime.datetime.strptime('2013-01-03', '%Y-%m-%d').date())))
]
# Split the data
y = data['incValue']
grippe = data['Grippe']
split = int(len(y) * 0.8)
y_train, y_test = y[:split], y[split:]
grippe_train, grippe_test = grippe[:split], grippe[split:]

i = 0
scores_df = pd.DataFrame(columns=['RMSE', 'lags', 'seasonal_lags', 'hidden_layers', 'alpha', 'batch_size', 'activation', 'learning_rate', 'exog_lags', 'grippe_lags'])

random.seed(42)
iterations = 500

# Grid search hyperparameter configurations

for _ in range(iterations):
    # Randomly select hyperparameters
    lag = random.randint(0, lags-20)
    activation = random.choice(activations)
    learning_rate = random.choice(learning_rates)
    alpha = random.choice(alphas)
    hidden_layer_size = random.choice(hidden_layer_sizes)
    exog_lag = random.choice(exog_lags)
    grippe_lag = random.randint(1, exog_lag)
    # Keep track of configurations and cv scores
    model = MLPRegressor(max_iter=1000, 
                        random_state=42, 
                        solver='adam', 
                        activation=activation, 
                        hidden_layer_sizes=hidden_layer_size, 
                        alpha=alpha, 
                        batch_size=batch_size, 
                        learning_rate_init=learning_rate,
                        warm_start=False, 
                        early_stopping=True)

    grippe_model = MLPRegressor(max_iter=2000, 
                        random_state=42, 
                        solver='adam', 
                        activation='relu', 
                        hidden_layer_sizes=(16, 16, 16), 
                        alpha=0.1343, 
                        batch_size=32, 
                        learning_rate_init=0.000413)

    scores = []

    # Define number of lags to create in the training set (exogenous variables config cross-validated separately)
    column_lags = {
        'incValue': lag,
        'Grippe': exog_lag
        }

    # Create lagged features based on the whole available training data
    df_lagged = create_lagged_features(pd.DataFrame({'incValue': y_train, 'Grippe': grippe_train}), column_lags=column_lags, seasonal_lags=[52])
    df_lagged.dropna(inplace=True)
    
    # Assign columns for preprocessing and training of autoregressive component
    flu_training_cols = [col for col in df_lagged.columns if ('incValue' in col) and ('lag' in col) and ('_helper' not in col)] # Select cols to include as features
    flu_seasonal_col = [col for col in df_lagged.columns if ('incValue' in col) and ('_helper' in col)] # Select cols to include as features
    X = df_lagged[flu_training_cols]
    X_seasonal = df_lagged[flu_seasonal_col]
    y = df_lagged['incValue']

    # Assign columns for preprocessing and training of exogenous component
    grippe_training_cols = [col for col in df_lagged.columns if ('Grippe' in col) and ('lag' in col) and ('_helper' not in col)] # Select cols to include as features
    grippe_seasonal_col = [col for col in df_lagged.columns if ('Grippe' in col) and ('_helper' in col)] # Select cols to include as features
    grippe = df_lagged[grippe_training_cols]
    grippe_seasonal = df_lagged[grippe_seasonal_col]
    grippe_y = df_lagged['Grippe']
    
    train_index = range(0, len(y) - 52)
    val_index = range(len(y) - 52, len(y))

        
    ##################################  TRANSFORMATION AND SCALING OF AUTOREGRESSIVE LAGS  #######################################################
    # 
    y_train_cv, y_val = y.iloc[train_index], y.iloc[val_index]
    X_train_cv, X_val = X.iloc[train_index], X.iloc[val_index]
    X_seasonal_train, X_seasonal_val = X_seasonal.iloc[train_index], X_seasonal.iloc[val_index]

    # Take the first row of X_train_cv (the oldest lags)
    oldest_lags = X_train_cv.iloc[0, 1:].values.reshape(1, -1)

    # Concatenate y_train_cv with the oldest lags
    combined_data = np.vstack((y_train_cv.values.reshape(-1, 1), oldest_lags.T))

    # Fit the PowerTransformer and StandardScaler on the available lags in the training data (incl. lags in first row of lag df_train)
    pt = PowerTransformer(method='yeo-johnson')
    stdscaler = StandardScaler()
    combined_data_transformed = pt.fit_transform(combined_data)
    stdscaler.fit(combined_data_transformed)
    
    # Apply Transform to the entire y_train_cv
    y_train_cv_transformed = pt.transform(y_train_cv.values.reshape(-1, 1)).flatten()
    y_val_transformed = pt.transform(y_val.values.reshape(-1, 1)).flatten()

    # Apply the PowerTransformer to each lagged feature in X_train_cv and X_val
    X_train_cv_transformed = X_train_cv.apply(lambda column: pt.transform(column.values.reshape(-1, 1)).flatten())
    X_val_transformed = X_val.apply(lambda column: pt.transform(column.values.reshape(-1, 1)).flatten())
    X_seasonal_train_trans = pt.transform(X_seasonal_train.values.reshape(-1, 1)).flatten()
    X_seasonal_val_trans = pt.transform(X_seasonal_val.values.reshape(-1, 1)).flatten()

    
    # Apply StandardScaler()
    y_train_cv_scaled = stdscaler.transform(y_train_cv_transformed.reshape(-1, 1)).flatten()
    y_val_scaled = stdscaler.transform(y_val_transformed.reshape(-1, 1)).flatten()
    X_train_cv_scaled = X_train_cv_transformed.apply(lambda column: stdscaler.transform(column.values.reshape(-1, 1)).flatten())
    X_val_scaled = X_val_transformed.apply(lambda column: stdscaler.transform(column.values.reshape(-1, 1)).flatten())
    X_seasonal_train_scaled = stdscaler.transform(X_seasonal_train_trans.reshape(-1, 1)).flatten()
    X_seasonal_val_scaled = stdscaler.transform(X_seasonal_val_trans.reshape(-1, 1)).flatten()

    #########################################  SCALE GOOGLE DATA  ##################################################
    # Scale Google Data
    grippe_y_train_cv, grippe_y_val = grippe_y.iloc[train_index], grippe_y.iloc[val_index]
    grippe_train_cv, grippe_val = grippe.iloc[train_index], grippe.iloc[val_index]
    grippe_seasonal_train, grippe_seasonal_val = grippe_seasonal.iloc[train_index], grippe_seasonal.iloc[val_index]
    # Take the first row of X_train_cv (the oldest lags)
    oldest_grippe_lags = grippe_train_cv.iloc[0, 1:].values.reshape(1, -1)
    # Concatenate grippe_y_train_cv with the oldest lags
    grippe_combined_data = np.vstack((grippe_y_train_cv.values.reshape(-1, 1), oldest_grippe_lags.T))
    
    # Fit the PowerTransformer and StandardScaler on the available lags in the training data (incl. lags in first row of lag df_train)
    grippe_pt = PowerTransformer(method='yeo-johnson')
    grippe_stdscaler = StandardScaler()
    grippe_combined_data_transformed = grippe_pt.fit_transform(grippe_combined_data)
    grippe_stdscaler.fit(grippe_combined_data_transformed)

    # Apply Transform to the entire y_train_cv
    grippe_y_train_cv_transformed = pt.transform(grippe_y_train_cv.values.reshape(-1, 1)).flatten()
    grippe_y_val_transformed = pt.transform(grippe_y_val.values.reshape(-1, 1)).flatten()

    # Apply the PowerTransformer to each lagged feature in X_train_cv and X_val
    grippe_train_cv_transformed = grippe_train_cv.apply(lambda column: grippe_pt.transform(column.values.reshape(-1, 1)).flatten())
    grippe_val_transformed = grippe_val.apply(lambda column: grippe_pt.transform(column.values.reshape(-1, 1)).flatten())
    grippe_seasonal_train_trans = grippe_pt.transform(grippe_seasonal_train.values.reshape(-1, 1)).flatten()
    grippe_seasonal_val_trans = grippe_pt.transform(grippe_seasonal_val.values.reshape(-1, 1)).flatten()

    # Apply StandardScaler()
    grippe_y_train_cv_scaled = grippe_stdscaler.transform(grippe_y_train_cv_transformed.reshape(-1, 1)).flatten()
    grippe_y_val_scaled = grippe_stdscaler.transform(grippe_y_val_transformed.reshape(-1, 1)).flatten()
    grippe_train_cv_scaled = grippe_train_cv_transformed.apply(lambda column: grippe_stdscaler.transform(column.values.reshape(-1, 1)).flatten())
    grippe_val_scaled = grippe_val_transformed.apply(lambda column: grippe_stdscaler.transform(column.values.reshape(-1, 1)).flatten())
    grippe_seasonal_train_scaled = grippe_stdscaler.transform(grippe_seasonal_train_trans.reshape(-1, 1)).flatten()
    grippe_seasonal_val_scaled = grippe_stdscaler.transform(grippe_seasonal_val_trans.reshape(-1, 1)).flatten()


    #######################################  FORECASTING GOOGLE DATA   ################################################################
    grippe_model.fit(grippe_train_cv_scaled.values, grippe_y_train_cv_scaled)

    grippe_input = create_exogenous_input(model=grippe_model, 
                                            initial_input=grippe_val_scaled.iloc[0], 
                                            seasonal_input=grippe_seasonal_val_scaled, 
                                            n_steps=len(grippe_y_val_scaled))
    
    grippe_input_on_test = create_exogenous_input(model=grippe_model, 
                                                    initial_input=grippe_train_cv_scaled.iloc[0], 
                                                    seasonal_input=grippe_seasonal_train_scaled, 
                                                    n_steps=len(grippe_y_train_cv_scaled))

    #########################################  PLOT VALIDATION LOSSES IF REQUIRED  ###########################################################
    # NOTE: PLOT VALIDATION AND TRAINING LOSSES - Adjust max_iter to 1 and set warm_start = True to enable

    # training_losses = []
    # validation_losses = []

    # for epoch in range(1000):  # Adjust the number of epochs as needed
    #     model.fit(X_train_cv_scaled.values, y_train_cv_scaled)

    #     # Store training loss from the last iteration
    #     training_losses.append(model.loss_curve_[-1])

    #     # Compute and store validation loss
    #     val_predictions = model.predict(X_val_scaled.values)
    #     val_loss = mean_squared_error(y_val_scaled, val_predictions)
    #     validation_losses.append(val_loss)
    
    # if fold == 2:
    #     plt.plot(training_losses, label='Training Loss')
    #     # If you have validation loss, plot it here
    #     plt.plot(validation_losses, label='Validation Loss')

    #     plt.title('Learning Curve')
    #     plt.xlabel('Epochs')
    #     plt.ylabel('Loss')
    #     plt.title(f'Lags: {lag}, Learning-rate: {learning_rate}, alpha: {alpha}, hidden layers: {hidden_layer_size}')
    #     plt.legend()
    #     plt.show()


    ########################################################  FIT MODEL AND MAKE INCVALUE FORECASTS  ####################################################################
    
    # Fit model with varying numbers of lags of the exogenous variable
    model.fit(np.concatenate(
        (
            X_train_cv_scaled.values, 
            grippe_train_cv_scaled.values[:, :grippe_lag],  
            grippe_train_cv_scaled.values[:, -1:]
        ), 
        axis=1), 
        y_train_cv_scaled
        )
    
    # Make iterative forecasts (NOTE: train and val splits are numpy arrays, seasonal helper columns necessary for updating of seasonal lag)
    prediction = iterative_forecast(model=model, 
                                    initial_input=X_val_scaled.iloc[0], 
                                    seasonal_input=X_seasonal_val_scaled, 
                                    exogenous_forecasts=np.concatenate((grippe_input[:, :grippe_lag], grippe_input[:, -1:]), axis=1), 
                                    n_steps=len(y_val_scaled))
    
    # y_hat_train = iterative_forecast(model=model, initial_input=X_train_cv_scaled.iloc[0], seasonal_input=X_seasonal_train_scaled, exogenous_forecasts=grippe_input, n_steps=len(y_train_cv_scaled))
    prediction = np.array(prediction).flatten()
    # y_hat_train = np.array(y_hat_train).flatten()

    #######################
    # NOTE: UNCOMMENT FOR PLOTS OF VALIDATION - Plot actual vs predicted values
    # plt.figure(figsize=(10, 6))
    # plt.plot(range(len(y_train_cv_scaled)), y_train_cv_scaled, label='Training Actual', color='blue')
    # plt.plot(range(len(y_train_cv_scaled), len(y_train_cv_scaled) + len(y_val_scaled)), y_val_scaled, label='Validation Actual', color='blue')
    # plt.plot(range(len(y_train_cv_scaled), len(y_train_cv_scaled) + len(y_val_scaled)), prediction, label='Validation Predicted', color='red', linestyle='--')
    # plt.plot(range(len(y_train_cv_scaled)), y_hat_train, label='Train-Set Prediction', color='orange', linestyle='--')
    
    # plt.title(f'Lag: {lag}; Predictions vs Actual')
    # plt.xlabel('Time')
    # plt.ylabel('Scaled Value')
    # plt.legend()
    # plt.show()
    ########################

    rmse = mean_squared_error(y_val_scaled, prediction, squared=False)
    scores.append(rmse)
    
    # Fill in parameters and score for each configuration 
    scores_df.loc[i, 'lags'] = lag
    scores_df.loc[i, 'seasonal_lags'] = seasonal
    scores_df.at[i, 'hidden_layers'] = hidden_layer_size
    scores_df.loc[i, 'alpha'] = alpha
    scores_df.loc[i, 'batch_size'] = batch_size
    scores_df.loc[i, 'activation'] = activation
    scores_df.loc[i, 'learning_rate'] = learning_rate
    scores_df.loc[i, 'exogenous_lags'] = exog_lag
    scores_df.loc[i, 'grippe_lag'] = grippe_lag
    scores_df.loc[i, 'RMSE'] = np.mean(scores)
    print(f'{i}/{iterations}: {(i/iterations)*100:.2f}%')
    i += 1

In [ ]:
scores_df['RMSE'] = pd.to_numeric(scores_df['RMSE'])
# Best parameters and score
best_config_index = scores_df['RMSE'].idxmin()  # This gets the index of the minimum RMSE
best_config = scores_df.loc[best_config_index]  # Use the index to access the row
best_score = best_config['RMSE']
print(f"Best parameters: {best_config}")
print(f"Best score (RMSE): {best_score}")

In [ ]:
best_config.values[3]

In [ ]:
scores_df.sort_values(by='RMSE').head(10)

<h2>Predict on Test Set</h2>

In [ ]:
rank_nr = 1

for index_nr in scores_df.sort_values(by='RMSE').head(10).index:
    best_config = scores_df.loc[index_nr]

    best_lag = best_config.values[1]
    best_seasonal_lag = best_config.values[2]
    best_hidden_layers = best_config.values[3]
    best_alpha = best_config.values[4]
    best_batch_size = best_config.values[5]
    best_activation = best_config.values[6]
    best_learning_rate = best_config.values[7]
    best_exogenous_lag = int(best_config.values[10])
    best_grippe_lag = int(best_config.values[11])

    # Filter the DataFrame
    data = merged_data.loc[
        (merged_data['georegion'] == "region_5") & 
        (merged_data['date'].apply(lambda x: (x.year < 2020))) & 
        (merged_data['date'].apply(lambda x: (x > datetime.datetime.strptime('2013-01-03', '%Y-%m-%d').date())))
    ]
    # Split the data
    y = data['incValue']
    grippe_y = data['Grippe']

    column_lags = {
                    'incValue': best_lag,
                    'Grippe': int(best_grippe_lag)
                    }
        
    # Create lagged features based on the whole available training data
    df_lagged = create_lagged_features(pd.DataFrame({'incValue': y, 'Grippe': grippe_y}), column_lags=column_lags, seasonal_lags=[52])

    split = int(len(y) * 0.8)
    # NOTE: SPLIT BEFORE DROPPING TO AVOID DATA LEAKAGE
    df_lagged_train = df_lagged.iloc[:split]
    df_lagged_train = df_lagged_train.dropna()
    df_lagged_test = df_lagged.iloc[split:]

    grippe_training_cols = [col for col in df_lagged.columns if ('Grippe' in col) and ('lag' in col) and ('_helper' not in col)] # Select cols to include as features
    grippe_train = df_lagged_train[grippe_training_cols]
    grippe_y_train = df_lagged_train['Grippe']
    grippe_test = df_lagged_test[grippe_training_cols]
    grippe_y_test = df_lagged_test['Grippe']

    # Columns required for rolling of seasonal lag in iterative autoregressive forecast
    grippe_seasonal_col = [col for col in df_lagged.columns if ('Grippe' in col) and ('_helper' in col)] # Select cols to include as features
    grippe_train_seasonal = df_lagged_train[grippe_seasonal_col]
    grippe_test_seasonal = df_lagged_test[grippe_seasonal_col]

    # Take the first row of X_train_cv (the oldest lags)
    oldest_grippe_lags = grippe_train.iloc[0, 1:].values.reshape(1, -1)
    # Concatenate grippe_y_train_cv with the oldest lags
    grippe_combined_data = np.vstack((grippe_y_train.values.reshape(-1, 1), oldest_grippe_lags.T))

    # Fit the PowerTransformer and StandardScaler on the available lags in the training data (incl. lags in first row of lag df_train)
    grippe_pt = PowerTransformer(method='yeo-johnson', standardize=False)
    grippe_stdscaler = StandardScaler()
    grippe_combined_data_transformed = grippe_pt.fit_transform(grippe_combined_data)
    grippe_stdscaler.fit(grippe_combined_data_transformed)

    # Apply Transform to the entire y_train
    grippe_y_train_transformed = grippe_pt.transform(grippe_y_train.values.reshape(-1, 1)).flatten()
    grippe_y_test_transformed = grippe_pt.transform(grippe_y_test.values.reshape(-1, 1)).flatten()

    # Apply the PowerTransformer to each lagged feature in X_train and X_test
    grippe_train_transformed = grippe_train.apply(lambda column: grippe_pt.transform(column.values.reshape(-1, 1)).flatten())
    grippe_test_transformed = grippe_test.apply(lambda column: grippe_pt.transform(column.values.reshape(-1, 1)).flatten())
    grippe_train_seasonal_trans = grippe_pt.transform(grippe_train_seasonal.values.reshape(-1, 1)).flatten()
    grippe_test_seasonal_trans = grippe_pt.transform(grippe_test_seasonal.values.reshape(-1, 1)).flatten()

    # Apply StandardScaler()
    grippe_y_train_scaled = grippe_stdscaler.transform(grippe_y_train_transformed.reshape(-1, 1)).flatten()
    grippe_y_test_scaled = grippe_stdscaler.transform(grippe_y_test_transformed.reshape(-1, 1)).flatten()
    grippe_train_scaled = grippe_train_transformed.apply(lambda column: grippe_stdscaler.transform(column.values.reshape(-1, 1)).flatten())
    grippe_test_scaled = grippe_test_transformed.apply(lambda column: grippe_stdscaler.transform(column.values.reshape(-1, 1)).flatten())
    grippe_train_seasonal_scaled = grippe_stdscaler.transform(grippe_train_seasonal_trans.reshape(-1, 1)).flatten()
    grippe_test_seasonal_scaled = grippe_stdscaler.transform(grippe_test_seasonal_trans.reshape(-1, 1)).flatten()

    # Initialize configuration of MLPRegressor based on CV results from separate training
    grippe_model = MLPRegressor(max_iter=2000, 
                                random_state=42, 
                                solver='adam', 
                                activation='relu', 
                                hidden_layer_sizes=(16, 16), 
                                alpha=0.2, 
                                batch_size=32, 
                                learning_rate_init=0.0001)

    grippe_model.fit(grippe_train_scaled.values, grippe_y_train_scaled)

    grippe_input = create_exogenous_input(model=grippe_model, initial_input=grippe_test_scaled.iloc[0], seasonal_input=grippe_test_seasonal_scaled, n_steps=len(grippe_y_test_scaled))
    grippe_input_train = create_exogenous_input(model=grippe_model, initial_input=grippe_train_scaled.iloc[0], seasonal_input=grippe_train_seasonal_scaled, n_steps=len(grippe_y_train_scaled))



    # Extract training columns and output variable from dataframe
    flu_training_cols = [col for col in df_lagged.columns if ('incValue' in col) and ('lag' in col) and ('_helper' not in col)] # Select cols to include as features
    X_train = df_lagged_train[flu_training_cols]
    y_train = df_lagged_train['incValue']
    X_test = df_lagged_test[flu_training_cols]
    y_test = df_lagged_test['incValue']

    # Columns required for rolling of seasonal lag in iterative autoregressive forecast
    flu_seasonal_col = [col for col in df_lagged.columns if ('incValue' in col) and ('_helper' in col)] # Select cols to include as features
    X_train_seasonal = df_lagged_train[flu_seasonal_col]
    X_test_seasonal = df_lagged_test[flu_seasonal_col]

    # Create combined data to fit transform on all available historical lags in training set
    oldest_lags = X_train.iloc[0, 1:].values.reshape(1, -1) # Take the first row of X_train (the oldest lags)
    combined_data = np.vstack((y_train.values.reshape(-1, 1), oldest_lags.T)) # Concatenate y_train_cv with the oldest lags

    # Fit Yeo-Johnson Transform on combined data
    pt = PowerTransformer(method='yeo-johnson', standardize=False)
    stdscaler = StandardScaler()
    combined_data_transformed = pt.fit_transform(combined_data)
    stdscaler.fit(combined_data_transformed)

    # Apply transform and scaling to train and test sets
    y_train_transformed = pt.transform(y_train.values.reshape(-1, 1)).flatten()
    y_test_transformed = pt.transform(y_test.values.reshape(-1, 1)).flatten()
    X_train_transformed = X_train.apply(lambda x: pt.transform(x.values.reshape(-1, 1)).flatten())
    X_test_transformed = X_test.apply(lambda x: pt.transform(x.values.reshape(-1, 1)).flatten())
    X_train_seasonal_trans = pt.transform(X_train_seasonal.values.reshape(-1, 1)).flatten()
    X_test_seasonal_trans = pt.transform(X_test_seasonal.values.reshape(-1, 1)).flatten()

    # Apply StandardScaler
    y_train_scaled = stdscaler.transform(y_train_transformed.reshape(-1, 1)).flatten()
    y_test_scaled = stdscaler.transform(y_test_transformed.reshape(-1, 1)).flatten()
    X_train_scaled = X_train_transformed.apply(lambda x: stdscaler.transform(x.values.reshape(-1, 1)).flatten())
    X_test_scaled = X_test_transformed.apply(lambda x: stdscaler.transform(x.values.reshape(-1, 1)).flatten())
    X_train_seasonal_scaled = stdscaler.transform(X_train_seasonal_trans.reshape(-1, 1)).flatten()
    X_test_seasonal_scaled = stdscaler.transform(X_test_seasonal_trans.reshape(-1, 1)).flatten()

    # Initialize the final model configuration
    final_model = MLPRegressor(max_iter=2000, 
                        random_state=42, 
                        solver='adam', 
                        activation=best_activation, 
                        hidden_layer_sizes=(best_hidden_layers), 
                        alpha=best_alpha, 
                        batch_size=best_batch_size, 
                        learning_rate_init=best_learning_rate)

    # Train final model
    final_model.fit(np.concatenate((X_train_scaled.values, grippe_train_scaled.values[:, :best_exogenous_lag], grippe_train_scaled.values[:, -1:]), axis=1), y_train_scaled)

    # Forecast for the length of the test set
    forecasts = iterative_forecast(model=final_model, 
                                initial_input=X_test_scaled.iloc[0], 
                                seasonal_input=X_test_seasonal_scaled, 
                                exogenous_forecasts=np.concatenate((grippe_input[:, :best_exogenous_lag], grippe_input[:, -1:]), axis=1), 
                                n_steps=len(y_test_scaled))

    y_hat_train = iterative_forecast(model=final_model, 
                                    initial_input=X_train_scaled.iloc[0], 
                                    seasonal_input=X_train_seasonal_scaled, 
                                    exogenous_forecasts=np.concatenate((grippe_input_train[:, :best_exogenous_lag], grippe_input_train[:, -1:]), axis=1), 
                                    n_steps=len(y_train_scaled))

    rmse = mean_squared_error(y_test_scaled, forecasts, squared=False)
    print(rmse)
    forecasts = stdscaler.inverse_transform(forecasts.reshape(-1, 1))
    forecasts = pt.inverse_transform(forecasts.reshape(-1, 1))
    y_hat_train = stdscaler.inverse_transform(y_hat_train.reshape(-1, 1))
    y_hat_train = pt.inverse_transform(y_hat_train.reshape(-1, 1))

    # Evaluate the forecasts against the actual y_test values
    rmse = mean_squared_error(y_test, forecasts, squared=False)

    # Plot the results
    fig, ax = plt.subplots(figsize=(15, 5))

    # Plot the true values
    # ax.plot(plot['incValue'])

    ax.plot(df_lagged['incValue'], label="True Train", alpha=1, color='lightblue')
    ax.plot(y_test.index, y_test, label="True Test", alpha=0.8, color='blue')
    ax.plot(y_test.index, forecasts, label='Predictions', alpha=0.7, color='red', linestyle='--')
    ax.plot(y_train.index, y_hat_train, label='Prediction on Train', alpha=0.7, color='grey', linestyle='--')


    # Add labels and legend
    ax.set_xlabel("Date")
    ax.set_ylabel("Incidence")
    ax.set_title(f'Nr: {rank_nr}, RMSE: {rmse}, auto_lag: {best_lag}, google_lag: {best_grippe_lag}, hidden layers: {best_hidden_layers}, alpha: {best_alpha:.4f}, learning rate: {best_learning_rate:.6f}', fontsize=10)
    ax.legend()
    rank_nr += 1

<h2>Discussion</h2>

As can be seen in the plots below, including the forecasted lags from the Google search trend data did not help our predictions on the test set and in many of the cases above resulted in much worse predictions. This may be due to an accumulation of errors in the iterative procedure from both models. Further tuning is likely required but was limited by computational and time constraints in this study.